In [ ]:
# Evaluating the c-VEP experiments data
import os
import numpy as np
import pyxdf
import pyntbci

In [11]:
KEY_MAPPING = {
    "slash": "/",
    "comma": ",",
    "colon": ":",
    "asterisk": "*",
    "question": "?",
    "quote": '"',
    "smaller": "<",
    "larger": ">",
    "bar": "|",
    "tilde": "~",
    "space": " ",
}

N_CLASSES = 63
ITI = 1

In [8]:
subjects = ["data_p001", "data_p002", "data_p003"]
runs = ["a", "b", "c"]
# data_dir = "/Users/u829215/Downloads/cvep_online"
data_dir = "."

In [12]:
cspm = []
p = []
tt = []
itr = []
for subject in subjects:
    print(subject)
    
    for run in runs:
        print(f"\t{run}")

        fn = os.path.join(data_dir, subject, f"sub-p001_ses-s001_run-001{run}_task-online.xdf")
        
        streams = pyxdf.load_xdf(fn)[0]
        names = [stream["info"]["name"][0] for stream in streams]
        stream = streams[names.index("cvep-speller-stream")]
        
        start_time = []
        stop_time = []
        predictions = []
        for timestamp, marker in zip(stream["time_stamps"], stream["time_series"]):
            if marker[0] == "start_trial":
                start_time.append(timestamp)
            elif "prediction" in marker[0]:
                stop_time.append(timestamp)
                predictions.append(marker[0].split('"')[-2])
        
        assert len(start_time) == len(stop_time)
        assert len(start_time) == len(predictions)

        predictions = [KEY_MAPPING[symbol] if symbol in KEY_MAPPING else symbol for symbol in predictions]

        print(f"\t\tPredictions: {"".join(predictions)}")
        
        print(f"\t\tNumber of trials: {len(start_time):d}")
        
        n_back = sum([symbol == "<" for symbol in predictions])
        print(f"\t\tNumber of backspaces: {n_back:d}")

        p.append((len(predictions) - n_back) / len(predictions))
        print(f"\t\tAccuracy: {p[-1]:.3f}")
        
        tt.append((np.array(stop_time) - np.array(start_time)).mean())
        print(f"\t\tAverage trial time: {tt[-1]:.3f} seconds")

        itr.append(pyntbci.utilities.itr(N_CLASSES, p[-1], tt[-1] + ITI)[0])
        print(f"\t\tITR: {itr[-1]:.3f}")
        
        sentence = ""
        for symbol in predictions:
            if symbol == "<":
                sentence = sentence[:-1]
            else:
                sentence += symbol
        print(f"\t\tSentence: {sentence}")
        
        print(f"\t\tSentence spelled in {(stop_time[-1] - start_time[0]) / 60:.3f} minutes")
        
        cspm.append(len(sentence) / ((stop_time[-1] - start_time[0]) / 60))
        print(f"\t\tCorrect symbols per minute: {cspm[-1]:.3f} symbols/minute")
            

data_p001
	a
		Predictions: !<THE QUICK BROWN FOX JU>*<MPS OVER :<C<THE LAZY DOGP<!!
		Number of trials: 56
		Number of backspaces: 5
		Accuracy: 0.911
		Average trial time: 3.423 seconds
		ITR: 67.988
		Sentence: THE QUICK BROWN FOX JU>MPS OVER THE LAZY DOG!!
		Sentence spelled in 4.142 minutes
		Correct symbols per minute: 11.105 symbols/minute
	b
		Predictions: !<THE Q @<<UICKZ< V<BROWN FOS<X JUMPS Q5<<OVER THE < LAZY D6<OG!!
		Number of trials: 65
		Number of backspaces: 10
		Accuracy: 0.846
		Average trial time: 3.412 seconds
		ITR: 60.402
		Sentence: THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG!!
		Sentence spelled in 4.799 minutes
		Correct symbols per minute: 9.376 symbols/minute
	c
		Predictions: !E@<[<<R^<<<2@<<TH+<E ?<QU <ICK BRK<Q<OWN FOX JU%<MPS OVER THZ<E LAZY D <OG!(<!
		Number of trials: 79
		Number of backspaces: 17
		Accuracy: 0.785
		Average trial time: 3.728 seconds
		ITR: 50.065
		Sentence: THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG!!
		Sentence spelled in 6.252 mi

In [13]:
print(f"Average correct symbols per minute: {sum(cspm) / len(cspm):.3f} symbols/minute")
print(f"Average accuracy: {sum(p) / len(p):.3f}")
print(f"Average trial time: {sum(tt) / len(tt):.3f} seconds")
print(f"Average ITR: {sum(itr) / len(itr):.3f}")

Average correct symbols per minute: 13.247 symbols/minute
Average accuracy: 0.895
Average trial time: 2.712 seconds
Average ITR: 81.942


In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "Subject": np.repeat(subjects, len(runs)),
        "Run": runs * len(subjects),
        "Correct symbols per minute": cspm,
        "Accuracy": p,
        "Average trial time [s]": tt,
        "ITR": itr,
    }
)
df.Subject = df.Subject.str.replace("data_", "")
df["Accuracy"] = df["Accuracy"].apply(lambda x: f"{x:.0%}")

ltx = df.to_latex(
    float_format="{:0.3f}".format,
    caption="Overview cVEP speller experiments",
    label="tab:cvep_results",
    index=False,
)

print(ltx)

\begin{table}
\caption{Overview cVEP speller experiments}
\label{tab:performance_comparison}
\begin{tabular}{llrlrr}
\toprule
Subject & Run & Correct symbols per minute & Accuracy & Average trial time [s] & ITR \\
\midrule
p001 & a & 11.105 & 91% & 3.423 & 67.988 \\
p001 & b & 9.376 & 85% & 3.412 & 60.402 \\
p001 & c & 7.198 & 78% & 3.728 & 50.065 \\
p002 & a & 14.829 & 91% & 2.268 & 91.149 \\
p002 & b & 17.291 & 94% & 2.048 & 104.407 \\
p002 & c & 17.444 & 94% & 2.021 & 105.332 \\
p003 & a & 15.089 & 94% & 2.495 & 91.055 \\
p003 & b & 11.652 & 85% & 2.547 & 75.137 \\
p003 & c & 15.240 & 94% & 2.469 & 91.940 \\
\bottomrule
\end{tabular}
\end{table}



In [42]:
dm = (
    df.assign(Accuracy=df.Accuracy.str.replace("%", "").astype("float") / 100)
    .groupby("Subject")[
        ["Correct symbols per minute", "Accuracy", "Average trial time [s]", "ITR"]
    ]
    .mean()
    .reset_index()
)
dm["Subject"] = [f"$\\text{{mean}}_{{\\text{{{s}}}}}$" for s in dm.Subject]
dm["Accuracy"] = dm["Accuracy"].apply(lambda x: f"{x:.0%}")

dm
ltx = dm.to_latex(
    float_format="{:0.3f}".format,
    index=False,
)

print(ltx)

\begin{tabular}{lrlrr}
\toprule
Subject & Correct symbols per minute & Accuracy & Average trial time [s] & ITR \\
\midrule
$\text{mean}_{\text{p001}}$ & 9.227 & 85% & 3.521 & 59.485 \\
$\text{mean}_{\text{p002}}$ & 16.522 & 93% & 2.113 & 100.296 \\
$\text{mean}_{\text{p003}}$ & 13.994 & 91% & 2.504 & 86.044 \\
\bottomrule
\end{tabular}



In [ ]:
mall = df.assign(Accuracy=df.Accuracy.str.replace("%", "").astype("float") / 100)[
    ["Correct symbols per minute", "Accuracy", "Average trial time [s]", "ITR"]
].mean()
dma = pd.DataFrame(mall).T
dma["Accuracy"] = dma["Accuracy"].apply(lambda x: f"{x:.0%}")
print(
    dma.to_latex(
        float_format="{:0.3f}".format,
        index=False,
    )
)

\begin{tabular}{rlrr}
\toprule
Correct symbols per minute & Accuracy & Average trial time [s] & ITR \\
\midrule
13.247 & 90% & 2.712 & 81.942 \\
\bottomrule
\end{tabular}

